In [1]:
import os
import sys
home_dir = "../../"
module_path = os.path.abspath(os.path.join(home_dir))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import pandas as pd
from models.aa_common.performance_metrics import *

In [2]:
task = "likely_pathogenic" # pathogenic, likely_pathogenic
model_root_and_name_tuple_list = [("dbnsfp", "sift"), ("dbnsfp", "polyphen2_HVAR"), ("dbnsfp", "cadd"), ("dbnsfp", "mvp"), ("dbnsfp", "metarnn"), ("dbnsfp", "revel"),
                                  ("tape_rao", "unirep"), ("tape_rao", "protbert"), ("sequnet_dunham", "sequnet"), 
                                  ("esm_rives", "esm1b_t33_650M_UR50S"), ("esm_rives", "esm1v_t33_650M_UR90S"), ("esm_rives", "esm2_t33_650M_UR50D"),
                                  ("bioembeddings_dallago", "plus_rnn"), 
                                  ("bioembeddings_dallago", "prottrans_bert_bfd"), ("bioembeddings_dallago", "prottrans_albert_bfd"),
                                  ("bioembeddings_dallago", "prottrans_xlnet_uniref100"), 
                                  ("bioembeddings_dallago", "prottrans_t5_bfd"), ("bioembeddings_dallago", "prottrans_t5_uniref50"), ("bioembeddings_dallago", "prottrans_t5_xl_u50")]

# by default, we consider smaller means damaging (here pathogenic or likely-pathogenic), but the following models have specific opposite meaning
models_larger_means_damaging = ["metarnn", "mvp", "polyphen2_HVAR", "cadd", "revel"]

metrics = ["AUC-ROC", "AUC-PR", "F1-max", "Th-max", "Precision", "Recall", "Accuracy", "Balanced-accuracy", "MCC"]

In [3]:
list_of_resut_df = []
for analysis_no in range(10):
    result_df = pd.read_csv(home_dir+f"models/aa_common/merged_predictions/{task}_analysis/{analysis_no}.csv", sep="\t")
    
    result_df.loc[result_df["class"]==task, "class_numeric"] = 0
    result_df.loc[result_df["class"]=="neutral", "class_numeric"] = 1
    
    list_of_resut_df.append(result_df)
    
print(list_of_resut_df[0].shape)
print(list_of_resut_df[0].columns)

(8336, 30)
Index(['id', 'chrom_acc_version', 'chrom_pos', 'ref_allele', 'alt_allele',
       'prot_acc_version', 'prot_pos', 'wt', 'mut', 'class', 'sift_pred',
       'polyphen2_HVAR_pred', 'cadd_pred', 'mvp_pred', 'metarnn_pred',
       'revel_pred', 'unirep_pred', 'protbert_pred', 'sequnet_pred',
       'esm1b_t33_650M_UR50S_pred', 'esm1v_t33_650M_UR90S_pred',
       'esm2_t33_650M_UR50D_pred', 'plus_rnn_pred', 'prottrans_bert_bfd_pred',
       'prottrans_albert_bfd_pred', 'prottrans_xlnet_uniref100_pred',
       'prottrans_t5_bfd_pred', 'prottrans_t5_uniref50_pred',
       'prottrans_t5_xl_u50_pred', 'class_numeric'],
      dtype='object')


In [4]:
performance_scores_sum_dict = {}

for analysis_no, result_df in enumerate(list_of_resut_df):
    output_file = home_dir+f"models/aa_common/performance_analysis/{task}_analysis/{analysis_no}.csv"

    out = open(output_file, 'w')
    out.write("Models\\Metrics")
    for metric in metrics:
        out.write(f"\t{metric}")
    out.write("\n")

    
    
    for i, (model_root, model_name) in enumerate(model_root_and_name_tuple_list):
        print(model_name)
        model_pred_col = model_name + "_pred"
        result_df["pred"]=(result_df[model_pred_col]-result_df[model_pred_col].min())/(result_df[model_pred_col].max()-result_df[model_pred_col].min()) # scaling prediction scores between [0, 1]
        
        non_nan_result_df = result_df[~pd.isna(result_df[model_pred_col])]  # taking df for only non-NAN values
        
        if model_name in models_larger_means_damaging:
            non_nan_result_df["pred"] = non_nan_result_df["pred"] * (-1)
        
        auc_roc_score = get_auc_roc_score(non_nan_result_df)
        auc_pr_score, precisions, recalls, thresholds = get_auc_pr_score(non_nan_result_df)
        f1_max, th_max = get_f1max_and_th(precisions, recalls, thresholds)
        precision = get_precision_score(non_nan_result_df, th_max)
        recall = get_recall_score(non_nan_result_df, th_max)
        accuracy = get_accuracy_score(non_nan_result_df, th_max)
        balanced_accuracy = get_balanced_accuracy_score(non_nan_result_df, th_max)
        mcc = get_matthews_corrcoef(non_nan_result_df, th_max)
        
        permformance_scores = [auc_roc_score, auc_pr_score, f1_max, th_max, precision, recall, accuracy, balanced_accuracy, mcc]
        
        out.write(f"{model_name}")
        for score in permformance_scores:
            out.write(f"\t{score:.3f}")
        out.write("\n")
            
        if model_name not in performance_scores_sum_dict:
            performance_scores_sum_dict[model_name] = np.array(permformance_scores)
        else: performance_scores_sum_dict[model_name] += np.array(permformance_scores)
        # if i==5: break

    out.close()

    
output_file = home_dir+f"models/aa_common/performance_analysis/{task}_analysis/avg.csv"
out = open(output_file, 'w')
out.write("Models\\Metrics")
for metric in metrics:
    out.write(f"\t{metric}")
out.write("\n")
    
for model_name, permformance_scores_sum in performance_scores_sum_dict.items():
    out.write(f"{model_name}")
    for score in permformance_scores_sum:
        out.write(f"\t{score/10:.3f}")
    out.write("\n")
out.close()

sift
	AUC-ROC: 0.798
	AUC-PR: 0.810
	Best F1-Score: 0.755 at threshold: 0.003
	Precision score: 0.714 at threshold: 0.003
	Recall score: 0.802 at threshold: 0.003
	Accuracy score: 0.731 at threshold: 0.003
	Balanced accuracy score: 0.729 at threshold: 0.003
	MCC score: 0.463 at threshold: 0.003
polyphen2_HVAR
	AUC-ROC: 0.803
	AUC-PR: 0.827
	Best F1-Score: 0.759 at threshold: -0.985
	Precision score: 0.695 at threshold: -0.985
	Recall score: 0.837 at threshold: -0.985
	Accuracy score: 0.716 at threshold: -0.985
	Balanced accuracy score: 0.707 at threshold: -0.985
	MCC score: 0.430 at threshold: -0.985
cadd
	AUC-ROC: 0.789
	AUC-PR: 0.819
	Best F1-Score: 0.725 at threshold: -0.582
	Precision score: 0.680 at threshold: -0.582
	Recall score: 0.777 at threshold: -0.582
	Accuracy score: 0.702 at threshold: -0.582
	Balanced accuracy score: 0.701 at threshold: -0.582
	MCC score: 0.406 at threshold: -0.582
mvp
	AUC-ROC: 0.789
	AUC-PR: 0.778
	Best F1-Score: 0.747 at threshold: -0.939
	Precision s

/tmp/ipykernel_2608609/120015408.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_nan_result_df["pred"] = non_nan_result_df["pred"] * (-1)
/tmp/ipykernel_2608609/120015408.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_nan_result_df["pred"] = non_nan_result_df["pred"] * (-1)
/tmp/ipykernel_2608609/120015408.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

	Recall score: 0.840 at threshold: -0.735
	Accuracy score: 0.764 at threshold: -0.735
	Balanced accuracy score: 0.762 at threshold: -0.735
	MCC score: 0.531 at threshold: -0.735
unirep
	AUC-ROC: 0.460
	AUC-PR: 0.453
	Best F1-Score: 0.667 at threshold: 0.079
	Precision score: 0.501 at threshold: 0.079
	Recall score: 0.999 at threshold: 0.079
	Accuracy score: 0.501 at threshold: 0.079
	Balanced accuracy score: 0.501 at threshold: 0.079
	MCC score: 0.026 at threshold: 0.079
protbert
	AUC-ROC: 0.510
	AUC-PR: 0.493
	Best F1-Score: 0.674 at threshold: 0.598
	Precision score: 0.518 at threshold: 0.598
	Recall score: 0.964 at threshold: 0.598
	Accuracy score: 0.533 at threshold: 0.598
	Balanced accuracy score: 0.533 at threshold: 0.598
	MCC score: 0.130 at threshold: 0.598
sequnet
	AUC-ROC: 0.368
	AUC-PR: 0.408
	Best F1-Score: 0.666 at threshold: 0.000
	Precision score: 0.500 at threshold: 0.000
	Recall score: 1.000 at threshold: 0.000
	Accuracy score: 0.500 at threshold: 0.000
	Balanced accur

/tmp/ipykernel_2608609/120015408.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_nan_result_df["pred"] = non_nan_result_df["pred"] * (-1)
/tmp/ipykernel_2608609/120015408.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_nan_result_df["pred"] = non_nan_result_df["pred"] * (-1)
/tmp/ipykernel_2608609/120015408.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

	Balanced accuracy score: 0.782 at threshold: -0.655
	MCC score: 0.564 at threshold: -0.655
unirep
	AUC-ROC: 0.460
	AUC-PR: 0.453
	Best F1-Score: 0.667 at threshold: 0.087
	Precision score: 0.501 at threshold: 0.087
	Recall score: 0.999 at threshold: 0.087
	Accuracy score: 0.501 at threshold: 0.087
	Balanced accuracy score: 0.501 at threshold: 0.087
	MCC score: 0.023 at threshold: 0.087
protbert
	AUC-ROC: 0.513
	AUC-PR: 0.496
	Best F1-Score: 0.674 at threshold: 0.551
	Precision score: 0.513 at threshold: 0.551
	Recall score: 0.985 at threshold: 0.551
	Accuracy score: 0.524 at threshold: 0.551
	Balanced accuracy score: 0.524 at threshold: 0.551
	MCC score: 0.124 at threshold: 0.551
sequnet
	AUC-ROC: 0.370
	AUC-PR: 0.411
	Best F1-Score: 0.666 at threshold: 0.000
	Precision score: 0.500 at threshold: 0.000
	Recall score: 1.000 at threshold: 0.000
	Accuracy score: 0.500 at threshold: 0.000
	Balanced accuracy score: 0.500 at threshold: 0.000
	MCC score: 0.000 at threshold: 0.000
esm1b_t33_6

...

	AUC-ROC: 0.903
	AUC-PR: 0.917
	Best F1-Score: 0.832 at threshold: -0.828
	Precision score: 0.823 at threshold: -0.828
	Recall score: 0.842 at threshold: -0.828
	Accuracy score: 0.827 at threshold: -0.828
	Balanced accuracy score: 0.827 at threshold: -0.828
	MCC score: 0.654 at threshold: -0.828
revel
	AUC-ROC: 0.845
	AUC-PR: 0.858
	Best F1-Score: 0.784 at threshold: -0.703
	Precision score: 0.757 at threshold: -0.703
	Recall score: 0.814 at threshold: -0.703
	Accuracy score: 0.769 at threshold: -0.703
	Balanced accuracy score: 0.768 at threshold: -0.703
	MCC score: 0.538 at threshold: -0.703
unirep
	AUC-ROC: 0.461
	AUC-PR: 0.454
	Best F1-Score: 0.667 at threshold: 0.041
	Precision score: 0.500 at threshold: 0.041
	Recall score: 1.000 at threshold: 0.041
	Accuracy score: 0.501 at threshold: 0.041
	Balanced accuracy score: 0.501 at threshold: 0.041
	MCC score: 0.026 at threshold: 0.041
protbert
	AUC-ROC: 0.515
	AUC-PR: 0.497
	Best F1-Score: 0.674 at threshold: 0.445
	Precision score: 0.

/tmp/ipykernel_2608609/120015408.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_nan_result_df["pred"] = non_nan_result_df["pred"] * (-1)
/tmp/ipykernel_2608609/120015408.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_nan_result_df["pred"] = non_nan_result_df["pred"] * (-1)
/tmp/ipykernel_2608609/120015408.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

	Accuracy score: 0.504 at threshold: 0.101
	Balanced accuracy score: 0.504 at threshold: 0.101
	MCC score: 0.039 at threshold: 0.101
protbert
	AUC-ROC: 0.515
	AUC-PR: 0.494
	Best F1-Score: 0.674 at threshold: 0.495
	Precision score: 0.513 at threshold: 0.495
	Recall score: 0.981 at threshold: 0.495
	Accuracy score: 0.526 at threshold: 0.495
	Balanced accuracy score: 0.526 at threshold: 0.495
	MCC score: 0.125 at threshold: 0.495
sequnet
	AUC-ROC: 0.370
	AUC-PR: 0.410
	Best F1-Score: 0.667 at threshold: 0.000
	Precision score: 0.500 at threshold: 0.000
	Recall score: 1.000 at threshold: 0.000
	Accuracy score: 0.500 at threshold: 0.000
	Balanced accuracy score: 0.500 at threshold: 0.000
	MCC score: 0.000 at threshold: 0.000
esm1b_t33_650M_UR50S
	AUC-ROC: 0.820
	AUC-PR: 0.810
	Best F1-Score: 0.755 at threshold: 0.449
	Precision score: 0.702 at threshold: 0.449
	Recall score: 0.817 at threshold: 0.449
	Accuracy score: 0.735 at threshold: 0.449
	Balanced accuracy score: 0.735 at threshold: 

/tmp/ipykernel_2608609/120015408.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_nan_result_df["pred"] = non_nan_result_df["pred"] * (-1)
/tmp/ipykernel_2608609/120015408.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_nan_result_df["pred"] = non_nan_result_df["pred"] * (-1)
/tmp/ipykernel_2608609/120015408.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

	Precision score: 0.676 at threshold: -0.596
	Recall score: 0.792 at threshold: -0.596
	Accuracy score: 0.703 at threshold: -0.596
	Balanced accuracy score: 0.701 at threshold: -0.596
	MCC score: 0.410 at threshold: -0.596
mvp
	AUC-ROC: 0.793
	AUC-PR: 0.782
	Best F1-Score: 0.746 at threshold: -0.938
	Precision score: 0.693 at threshold: -0.938
	Recall score: 0.807 at threshold: -0.938
	Accuracy score: 0.726 at threshold: -0.938
	Balanced accuracy score: 0.727 at threshold: -0.938
	MCC score: 0.460 at threshold: -0.938
metarnn
	AUC-ROC: 0.908
	AUC-PR: 0.920
	Best F1-Score: 0.834 at threshold: -0.830
	Precision score: 0.820 at threshold: -0.830
	Recall score: 0.849 at threshold: -0.830
	Accuracy score: 0.829 at threshold: -0.830
	Balanced accuracy score: 0.829 at threshold: -0.830
	MCC score: 0.659 at threshold: -0.830
revel
	AUC-ROC: 0.859
	AUC-PR: 0.864
	Best F1-Score: 0.795 at threshold: -0.617
	Precision score: 0.808 at threshold: -0.617
	Recall score: 0.783 at threshold: -0.617
	Acc

In [5]:
import random

model_name = "random_classifier"
auc_roc_score_list, auc_pr_score_list, f1_max_list, th_max_list, precision_list, recall_list, accuracy_list, balanced_accuracy_list, mcc_list = [], [], [], [], [], [], [], [], []

for i in range(10):
    result_df["preds"] = [random.uniform(0, 1) for i in range(result_df.shape[0])]
    non_nan_result_df = result_df
        
    auc_roc_score = get_auc_roc_score(non_nan_result_df)
    auc_pr_score, precisions, recalls, thresholds = get_auc_pr_score(non_nan_result_df)
    f1_max, th_max = get_f1max_and_th(precisions, recalls, thresholds)
    precision = get_precision_score(non_nan_result_df, th_max)
    recall = get_recall_score(non_nan_result_df, th_max)
    accuracy = get_accuracy_score(non_nan_result_df, th_max)
    balanced_accuracy = get_balanced_accuracy_score(non_nan_result_df, th_max)
    mcc = get_matthews_corrcoef(non_nan_result_df, th_max)

    auc_roc_score_list.append(auc_roc_score)
    auc_pr_score_list.append(auc_pr_score)
    f1_max_list.append(f1_max)
    th_max_list.append(th_max)
    precision_list.append(precision)
    recall_list.append(recall)
    accuracy_list.append(accuracy)
    balanced_accuracy_list.append(balanced_accuracy)
    mcc_list.append(mcc)
    

permformance_scores = [np.mean(auc_roc_score_list), np.mean(auc_pr_score_list), np.mean(f1_max_list), 
                       np.mean(th_max_list), np.mean(precision_list), np.mean(recall_list), 
                       np.mean(accuracy_list), np.mean(balanced_accuracy_list), np.mean(mcc_list)]

output_file = home_dir+f"models/aa_common/performance_analysis/{task}_analysis/avg.csv"    
# output_file = home_dir+f"models/aa_common/performance_analysis/{task}.csv"
out = open(output_file, 'a')

out.write(f"{model_name}")
for score in permformance_scores:
    out.write(f"\t{score:.3f}")
out.write("\n")
out.close()

	AUC-ROC: 0.483
	AUC-PR: 0.486
	Best F1-Score: 0.667 at threshold: 0.125
	Precision score: 0.501 at threshold: 0.125
	Recall score: 0.998 at threshold: 0.125
	Accuracy score: 0.502 at threshold: 0.125
	Balanced accuracy score: 0.502 at threshold: 0.125
	MCC score: 0.030 at threshold: 0.125
	AUC-ROC: 0.483
	AUC-PR: 0.486
	Best F1-Score: 0.667 at threshold: 0.125
	Precision score: 0.501 at threshold: 0.125
	Recall score: 0.998 at threshold: 0.125
	Accuracy score: 0.502 at threshold: 0.125
	Balanced accuracy score: 0.502 at threshold: 0.125
	MCC score: 0.030 at threshold: 0.125
	AUC-ROC: 0.483
	AUC-PR: 0.486
	Best F1-Score: 0.667 at threshold: 0.125
	Precision score: 0.501 at threshold: 0.125
	Recall score: 0.998 at threshold: 0.125
	Accuracy score: 0.502 at threshold: 0.125
	Balanced accuracy score: 0.502 at threshold: 0.125
	MCC score: 0.030 at threshold: 0.125
	AUC-ROC: 0.483
	AUC-PR: 0.486
	Best F1-Score: 0.667 at threshold: 0.125
	Precision score: 0.501 at threshold: 0.125
	Recall sc